In [10]:
# ==============================================================================
# --- PHASE 0: LIBRARYS NEEDED  ---
# ==============================================================================

# Colab / System Management (Required for environment setup)
from google.colab import drive
import os
import zipfile

# Data Science and Processing
import pandas as pd
import numpy as np
import json
import re
import random

# NLP & Topic Modeling
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer # For Phase 6

# Installation for textstat must happen before import as it's not a default Colab package
print("Installing textstat for readability analysis...")
!pip install textstat -q
import textstat # For Phase 6

from sklearn.feature_extraction.text import CountVectorizer # For Phase 9
from sklearn.decomposition import LatentDirichletAllocation # For Phase 9

# Visualization
import matplotlib.pyplot as plt
from wordcloud import WordCloud # For Phase 8
from IPython.display import display, Markdown # FIX: Ensures the Markdown object is defined for displaying tables

# --- NLTK Setup ---
try:
    nltk.download('punkt', quiet=True)
    nltk.download('vader_lexicon', quiet=True)
except Exception:
    pass

# --- Global Parameters ---
DRIVE_PATH = '/content/drive/MyDrive'
ZIP_FILE_NAME = 'Bible-kjv-master.zip'
ZIP_FILE_PATH = os.path.join(DRIVE_PATH, ZIP_FILE_NAME)
EXTRACT_DIR = '/content/bible_data'
FINAL_DATA_DIR = '' # Will be set after successful unzipping

# --- Helper Function: Load Text for Specific Books ---
def load_book_texts(data_dir, book_names=None):
    """Loads raw text for specified books and returns a dictionary."""
    if not data_dir: return {}

    book_texts = {}

    # Generate expected filenames. If book_names is None, it loads all files.
    json_files = [f + '.json' if not f.endswith('.json') else f for f in book_names] if book_names else [f for f in os.listdir(data_dir) if f.endswith('.json')]

    for filename in json_files:
        filepath = os.path.join(data_dir, filename)
        full_book_text = []
        book_name = filename.replace('.json', '')

        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                book_data = json.load(f)

            book_info = book_data[0] if isinstance(book_data, list) and len(book_data) == 1 else book_data

            chapters_list = book_info.get('chapters', [])
            for chapter_obj in chapters_list:
                for verse_obj in chapter_obj.get('verses', []):
                    full_book_text.append(verse_obj.get('text', ''))

            # The key should match the book name used in the DataFrame
            if book_info.get('book'):
                book_name = book_info.get('book')

            book_texts[book_name] = " ".join(full_book_text)

        except Exception:
            continue

    return book_texts

Installing textstat for readability analysis...


In [12]:
# ==============================================================================
# --- PHASE 1: MOUNT GOOGLE DRIVE ---
# ==============================================================================
print("Mounting Google Drive...")
drive.mount('/content/drive')
print(f"Drive mounted. Check files in: {DRIVE_PATH}")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted. Check files in: /content/drive/MyDrive


In [13]:
# ==============================================================================
# --- PHASE 2: UNZIP THE BIBLE DATA (CONFIRMED PATH) ---
# ==============================================================================

# Create a destination directory
os.makedirs(EXTRACT_DIR, exist_ok=True)

print(f"Unzipping {ZIP_FILE_PATH} to {EXTRACT_DIR}...")
try:
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)

    temp_data_dir = os.path.join(EXTRACT_DIR, 'Bible-kjv-master')
    test_file = os.path.join(temp_data_dir, 'Genesis.json')

    if os.path.exists(test_file):
        FINAL_DATA_DIR = temp_data_dir
    else:
        FINAL_DATA_DIR = EXTRACT_DIR

    if os.path.exists(os.path.join(FINAL_DATA_DIR, 'Genesis.json')):
        print(f"✅ Success! Individual book JSON files found in: {FINAL_DATA_DIR}")
    else:
        print(f"❌ FATAL ERROR: Cannot confirm path to individual JSON files.")

except Exception as e:
    print(f"❌ FATAL ERROR during unzipping: {e}")
    FINAL_DATA_DIR = ''

Unzipping /content/drive/MyDrive/Bible-kjv-master.zip to /content/bible_data...
✅ Success! Individual book JSON files found in: /content/bible_data/Bible-kjv-master


In [14]:
# ==============================================================================
# --- PHASE 3: ANALYSIS (Core Metrics: Word Count, Holiness Focus Score) ---
# --- Using Robust REGEX for Holy/Holiness Counting ---
# ==============================================================================

def create_comprehensive_analysis_table_fixed(data_dir):
    """Calculates all metrics using robust regex for accurate word and theme counts."""
    if not data_dir: return pd.DataFrame()
    data = []

    try:
        json_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.json')])
    except FileNotFoundError:
        return pd.DataFrame()

    print(f"\n--- PHASE 5: Robust REGEX Counting ---")
    print(f"Found {len(json_files)} book files. Calculating all metrics...")

    # Robust patterns
    HOLY_PATTERN = re.compile(r'\b(holy|holiness)\b') # Catching both words robustly

    for filename in json_files:
        filepath = os.path.join(data_dir, filename)
        full_book_text = []

        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                book_data = json.load(f)

            book_info = book_data[0] if isinstance(book_data, list) and len(book_data) == 1 else book_data

            chapters_list = book_info.get('chapters', [])
            for chapter_obj in chapters_list:
                for verse_obj in chapter_obj.get('verses', []):
                    full_book_text.append(verse_obj.get('text', ''))

            book_text = " ".join(full_book_text).lower()

            # --- CALCULATION 1: Robust Total Word Count ---
            total_words = len(book_text.split()) # Simple whitespace split is standard for word count

            # --- CALCULATION 2: Robust Holiness Focus Score (Raw Count) ---
            holy_count = len(HOLY_PATTERN.findall(book_text))

            if total_words > 0:
                book_name = book_info.get('book', filename.replace('.json', ''))

                data.append({
                    'Book': book_name,
                    'Total Words': total_words,
                    'Holiness Focus Score': holy_count,
                    'Holiness Intensity': (holy_count / total_words) * 1000 # Count per 1000 words
                })

        except Exception:
            continue

    df = pd.DataFrame(data)

    df_sorted = df.sort_values(by='Holiness Intensity', ascending=False).reset_index(drop=True)

    # --- ADDED: Display Table for Phase 3 ---
    print("\n## 📊 PHASE 5: Holiness Intensity Ranking (Top 10 Books)")
    display_df = df_sorted[['Book', 'Total Words', 'Holiness Focus Score', 'Holiness Intensity']].head(30)

    # 🌟 NEW DEBUG STEP: Print raw DataFrame to guarantee output visibility
    print("--- Raw DataFrame Preview (Confirming Data Exists) ---")
    print(display_df)
    print("-------------------------------------------------------")

    display(Markdown(display_df.to_markdown(index=False, floatfmt=".2f")))
    # --- END ADDITION ---

    print("✅ Core metrics calculated. Proceeding to Phase 4.")

    return df_sorted

In [15]:
# --- EXECUTION PHASE 3 ---
FINAL_ANALYSIS_DF = pd.DataFrame()
if FINAL_DATA_DIR:
    FINAL_ANALYSIS_DF = create_comprehensive_analysis_table_fixed(FINAL_DATA_DIR)


--- PHASE 5: Robust REGEX Counting ---
Found 67 book files. Calculating all metrics...

## 📊 PHASE 5: Holiness Intensity Ranking (Top 10 Books)
--- Raw DataFrame Preview (Confirming Data Exists) ---
               Book  Total Words  Holiness Focus Score  Holiness Intensity
0           2 Peter         1553                     6            3.863490
1         Leviticus        24540                    94            3.830481
2   1 Thessalonians         1837                     7            3.810561
3             Titus          896                     3            3.348214
4              Jude          608                     2            3.289474
5           1 Peter         2476                     8            3.231018
6           Obadiah          669                     2            2.989537
7         Ephesians         3022                     7            2.316347
8              Acts        24245                    53            2.186018
9          Habakkuk         1475                  

| Book            |   Total Words |   Holiness Focus Score |   Holiness Intensity |
|:----------------|--------------:|-----------------------:|---------------------:|
| 2 Peter         |          1553 |                      6 |                 3.86 |
| Leviticus       |         24540 |                     94 |                 3.83 |
| 1 Thessalonians |          1837 |                      7 |                 3.81 |
| Titus           |           896 |                      3 |                 3.35 |
| Jude            |           608 |                      2 |                 3.29 |
| 1 Peter         |          2476 |                      8 |                 3.23 |
| Obadiah         |           669 |                      2 |                 2.99 |
| Ephesians       |          3022 |                      7 |                 2.32 |
| Acts            |         24245 |                     53 |                 2.19 |
| Habakkuk        |          1475 |                      3 |                 2.03 |
| 2 Timothy       |          1666 |                      3 |                 1.80 |
| Exodus          |         32684 |                     58 |                 1.77 |
| Haggai          |          1130 |                      2 |                 1.77 |
| Hebrews         |          6897 |                     12 |                 1.74 |
| Romans          |          9422 |                     16 |                 1.70 |
| Isaiah          |         37040 |                     61 |                 1.65 |
| Jonah           |          1320 |                      2 |                 1.52 |
| Joel            |          2033 |                      3 |                 1.48 |
| Daniel          |         11602 |                     17 |                 1.47 |
| Revelation      |         11995 |                     16 |                 1.33 |
| Ezekiel         |         39402 |                     49 |                 1.24 |
| Psalms          |         42685 |                     50 |                 1.17 |
| Colossians      |          1979 |                      2 |                 1.01 |
| Nehemiah        |         10480 |                     10 |                 0.95 |
| 1 Timothy       |          2244 |                      2 |                 0.89 |
| Numbers         |         32893 |                     29 |                 0.88 |
| 1 Corinthians   |          9462 |                      8 |                 0.85 |
| 1 John          |          2516 |                      2 |                 0.79 |
| Zechariah       |          6443 |                      5 |                 0.78 |
| Luke            |         25939 |                     20 |                 0.77 |

✅ Core metrics calculated. Proceeding to Phase 4.


In [16]:
# ==============================================================================
# --- PHASE 4: SENTIMENT & CONTRAST ANALYSIS (Righteousness, Sentiment) ---
# ==============================================================================

def run_sentiment_analysis(df_holiness, data_dir):
    """Calculates sentiment scores and 'Righteous' contrast for each book."""
    if df_holiness.empty or not data_dir: return df_holiness

    print("\n--- PHASE 4: Running Sentiment Analysis and Righteousness Contrast ---")

    book_names = df_holiness['Book'].tolist()
    # Load all book texts at once for efficiency
    book_texts_dict = load_book_texts(data_dir, book_names)

    sentiment_data = []
    sia = SentimentIntensityAnalyzer()
    RIGHTEOUS_PATTERN = re.compile(r'\b(righteous|righteousness)\b') # Robustly match both forms

    for book_name, text in book_texts_dict.items():
        # Get total words from the main DF for accurate normalization
        total_words = df_holiness[df_holiness['Book'] == book_name]['Total Words'].iloc[0]

        if not text or total_words == 0: continue

        text_lower = text.lower()

        # 1. Sentiment (VADER)
        vs = sia.polarity_scores(text)

        # 2. Thematic Contrast: 'Righteous' count & intensity
        righteous_count = len(RIGHTEOUS_PATTERN.findall(text_lower))
        righteous_intensity = (righteous_count / total_words) * 1000 if total_words > 0 else 0

        sentiment_data.append({
            'Book': book_name,
            'Sentiment (Compound)': vs['compound'],
            'Righteous Count (Raw)': righteous_count,
            'Righteous Intensity (Per 1000 Words)': righteous_intensity
        })

    df_sentiment = pd.DataFrame(sentiment_data)

    # Merge with the comprehensive analysis table (Phase 5 output)
    df_final = pd.merge(df_holiness, df_sentiment, on='Book', how='left')

    # Re-sort to maintain order after merge
    df_final = df_final.sort_values(by='Holiness Intensity', ascending=False).reset_index(drop=True)

    print("✅ Sentiment and Righteousness contrast complete.")

    # Print the resulting table (Top 10 for review)
    print("\n## 📚 PHASE 4: Comprehensive Book Analysis - Top 10 Metrics (Includes Sentiment)")
    display(Markdown(df_final.head(10).to_markdown(index=False, floatfmt=".4f")))

    # NEW DEBUG STEP: Print columns to confirm names match for Phase 7
    print(f"--- Phase 4 Final DataFrame Columns Confirmation ---")
    print(df_final.columns.tolist())
    print("----------------------------------------------------")

    return df_final

In [17]:
# --- EXECUTION PHASE 4 ---
if not FINAL_ANALYSIS_DF.empty:
    FINAL_ANALYSIS_DF = run_sentiment_analysis(FINAL_ANALYSIS_DF, FINAL_DATA_DIR)


--- PHASE 4: Running Sentiment Analysis and Righteousness Contrast ---
✅ Sentiment and Righteousness contrast complete.

## 📚 PHASE 4: Comprehensive Book Analysis - Top 10 Metrics (Includes Sentiment)


| Book            |   Total Words |   Holiness Focus Score |   Holiness Intensity |   Sentiment (Compound) |   Righteous Count (Raw) |   Righteous Intensity (Per 1000 Words) |
|:----------------|--------------:|-----------------------:|---------------------:|-----------------------:|------------------------:|---------------------------------------:|
| 2 Peter         |          1553 |                      6 |               3.8635 |               nan      |                nan      |                               nan      |
| Leviticus       |         24540 |                     94 |               3.8305 |                -0.9999 |                  1.0000 |                                 0.0407 |
| 1 Thessalonians |          1837 |                      7 |               3.8106 |               nan      |                nan      |                               nan      |
| Titus           |           896 |                      3 |               3.3482 |                 0.9995 |                  1.0000 |                                 1.1161 |
| Jude            |           608 |                      2 |               3.2895 |                 0.9935 |                  0.0000 |                                 0.0000 |
| 1 Peter         |          2476 |                      8 |               3.2310 |               nan      |                nan      |                               nan      |
| Obadiah         |           669 |                      2 |               2.9895 |                -0.9571 |                  0.0000 |                                 0.0000 |
| Ephesians       |          3022 |                      7 |               2.3163 |                 0.9999 |                  3.0000 |                                 0.9927 |
| Acts            |         24245 |                     53 |               2.1860 |                 1.0000 |                  4.0000 |                                 0.1650 |
| Habakkuk        |          1475 |                      3 |               2.0339 |                -0.9983 |                  2.0000 |                                 1.3559 |

--- Phase 4 Final DataFrame Columns Confirmation ---
['Book', 'Total Words', 'Holiness Focus Score', 'Holiness Intensity', 'Sentiment (Compound)', 'Righteous Count (Raw)', 'Righteous Intensity (Per 1000 Words)']
----------------------------------------------------


In [18]:
# ==============================================================================
# --- PHASE 5: PLOT GENERATION (ORGANIZED AS REQUESTED) ---
# 1. Holiness Intensity Ranking (Vertical, Descending, Top 30)
# 2. Holiness vs. Righteousness Comparison (Top 15)
# 3. Sentiment Compound Score (Top 10)
# ==============================================================================

def create_initial_analysis_plots(df_data):
    """Generates the requested plots in the specified order and format."""
    print("\n--- PHASE 5: Generating Reorganized Analysis Plots ---")

    # 🌟 NEW ROBUSTNESS CHECK: Verify required columns exist before proceeding
    required_cols = ['Holiness Intensity', 'Righteous Intensity (Per 1000 Words)', 'Sentiment (Compound)']

    if df_data.empty or not all(col in df_data.columns for col in required_cols):
        print("❌ FAILED TO PLOT: Cannot proceed with Phase 7 plots. The required data columns (Holiness Intensity, Righteous Intensity, or Sentiment) are missing.")
        if not df_data.empty:
            print(f"Found columns: {df_data.columns.tolist()}")
        print("💡 Action required: Check the output of Phase 6 to ensure it successfully loaded data and merged the 'Righteousness' and 'Sentiment' metrics.")
        return

    # Select the top 30 books for the main plot
    df_plot_top30 = df_data.head(30).copy()

    # Select the top 15 books for the comparison plot
    df_plot_top15 = df_data.head(15).copy()

    # Convert necessary columns to numeric for plotting (This is where the KeyError occurred previously)
    df_plot_top30['Holiness Intensity_Num'] = df_plot_top30['Holiness Intensity'].astype(float)
    df_plot_top15['Holiness Intensity_Num'] = df_plot_top15['Holiness Intensity'].astype(float)

    # ERROR FIX: This line will now only execute if the column check passed above
    df_plot_top15['Righteous Intensity_Num'] = df_plot_top15['Righteous Intensity (Per 1000 Words)'].astype(float)
    df_plot_top15['Sentiment (Compound)_Num'] = df_plot_top15['Sentiment (Compound)'].astype(float)


    # --- 1. PLOT: Holiness Intensity Ranking (Vertical, Descending) - TOP 30 ---
    fig, ax = plt.subplots(figsize=(20, 8)) # Increased size to accommodate 30 books

    books = df_plot_top30['Book']
    intensity = df_plot_top30['Holiness Intensity_Num']

    # Create a vertical bar chart
    ax.bar(books, intensity, color='#8B4513')

    ax.set_title('1. Holiness Intensity Map (Mentions of "Holy/Holiness" per 1000 Words) - Top 30 Books', fontsize=18, weight='bold')
    ax.set_xlabel('Book of the Bible (Highest Intensity on Left)', fontsize=14)
    ax.set_ylabel('Holiness Intensity (Count per 1000 Words)', fontsize=14)

    ax.set_xticklabels(books, rotation=60, ha="right", fontsize=10) # Rotate and shrink font for 30 labels
    ax.grid(axis='y', linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.savefig('Holiness_Intensity_Ranking_Vertical_Map_Top30.png')
    print("✅ Saved 1. Holiness Intensity Ranking Bar Chart (Vertical, Descending, Top 30).")
    plt.close()


    # --- 2. PLOT: Holiness Intensity vs. Righteous Intensity (Normalized Bar Chart - Top 15) ---
    fig, ax = plt.subplots(figsize=(18, 8))

    width = 0.35
    x = np.arange(len(df_plot_top15['Book']))

    # Holiness Intensity (Per 1000 Words)
    ax.bar(x - width/2, df_plot_top15['Holiness Intensity_Num'], width, label='Holy/Holiness Intensity', color='#4c72b0', alpha=0.9)

    # Righteous Intensity (Per 1000 Words)
    ax.bar(x + width/2, df_plot_top15['Righteous Intensity_Num'], width, label='Righteousness Intensity', color='#c44e52', alpha=0.9)

    ax.set_ylabel('Intensity (Mentions per 1000 Words)', fontsize=14)
    ax.set_title('2. Top 15 Books: Holy/Holiness vs. Righteousness Intensity Comparison', fontsize=16, weight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(df_plot_top15['Book'], rotation=45, ha="right")
    ax.legend(loc='upper right')
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('Holiness_Righteousness_Intensity_Comparison.png')
    print("✅ Saved 2. Holy/Righteousness Intensity Comparison Bar Chart (Top 15).")
    plt.close()


    # --- 3. PLOT: Sentiment (VADER Compound Score) for Top 10 Books ---
    df_plot_top10 = df_plot_top15.head(10).copy() # Filter to top 10 as requested

    fig, ax = plt.subplots(figsize=(16, 7))

    sentiment = df_plot_top10['Sentiment (Compound)_Num']

    # Color the bars based on the score
    colors = ['#4daf4a' if s > 0.05 else ('#e41a1c' if s < -0.05 else '#ff7f00') for s in sentiment]

    ax.bar(df_plot_top10['Book'], sentiment, color=colors, alpha=0.8)

    ax.axhline(0, color='gray', linestyle='--', linewidth=1.5) # Add zero line for reference

    ax.set_title('3. Top 10 Books: VADER Compound Sentiment Score', fontsize=16, weight='bold')
    ax.set_xlabel('Book', fontsize=12)
    ax.set_ylabel('VADER Compound Sentiment Score', fontsize=12)
    ax.set_xticklabels(df_plot_top10['Book'], rotation=45, ha="right")
    ax.grid(axis='y', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.savefig('Sentiment_Compound_Top10_Bar_Chart.png')
    print("✅ Saved 3. Sentiment Compound Score Bar Chart (Top 10).")
    plt.close()


In [19]:
# --- EXECUTION PHASE 5 ---
if not FINAL_ANALYSIS_DF.empty:
    create_initial_analysis_plots(FINAL_ANALYSIS_DF)


--- PHASE 5: Generating Reorganized Analysis Plots ---


/tmp/ipython-input-3165597962.py:50: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(books, rotation=60, ha="right", fontsize=10) # Rotate and shrink font for 30 labels


✅ Saved 1. Holiness Intensity Ranking Bar Chart (Vertical, Descending, Top 30).
✅ Saved 2. Holy/Righteousness Intensity Comparison Bar Chart (Top 15).


/tmp/ipython-input-3165597962.py:100: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(df_plot_top10['Book'], rotation=45, ha="right")


✅ Saved 3. Sentiment Compound Score Bar Chart (Top 10).


In [20]:
# ==============================================================================
# --- PHASE 6: BUBBLE PLOT IN THE 15 PRINCIPLE BOOKS (NEW) ---
# Visualizing Thematic Intensity (Mock) vs. Topic (Size=Total Words)
# ==============================================================================

def create_thematic_bubble_plot(df_data, data_dir):
    """Generates a thematic bubble plot for the top 15 books."""
    if df_data.empty:
        print("❌ Cannot run Phase 8: Input DataFrame is empty.")
        return

    print("\n--- PHASE 6: Generating Thematic Bubble Plot Visualization ---")

    # 1. Prepare Data (using REAL data and filtering for top 15)
    # Filter down to the top 15 books by Holiness Intensity
    df_top_15 = df_data.sort_values(by='Holiness Intensity', ascending=False).head(15).copy()

    if df_top_15.empty:
        print("❌ Cannot run Phase 8: Top 15 books list is empty after filtering.")
        return

    # Dynamically generate the required data structures
    top_15_books = df_top_15['Book'].tolist()
    word_counts_map = dict(zip(df_top_15['Book'], df_top_15['Total Words']))

    # 2. Topics (8 Topics - Static, as per original mock script)
    # NOTE: These are the exact topic titles requested by the user for this plot.
    topics = [
        'End Times', 'Prophecy/Judgment', 'Apostolic Teaching',
        'Salvation/Grace', 'History/Narrative', 'Kingdom/Rule',
        'Purity/Ritual', 'Mission/Evangelism'
    ]
    num_topics = len(topics)

    # 3. Bubble Size and Mock Data Generation (Using REAL Word Counts)
    word_counts = np.array([word_counts_map[book] for book in top_15_books])

    # Normalize to a visible range (50 to 5050)
    bubble_size = (word_counts / word_counts.max()) * 5000 + 50

    mock_data = []

    for book in top_15_books:
        topic_id = 0
        intensity_value = random.uniform(0.5, 1.5)

        # Logic to assign a primary topic and relevant intensity (same logic as user's original mock script)
        if book in ['2 Peter', '1 Thessalonians', 'Titus', '2 Timothy', '1 Peter', 'Ephesians']:
            # Apostolic Teaching (2), Salvation/Grace (3), Mission/Evangelism (7)
            topic_id = random.choice([2, 3, 7])
            intensity_value = random.uniform(3.8, 5.0)
        elif book in ['Jude', 'Obadiah', 'Habakkuk', 'Jonah']:
            # End Times (0), Prophecy/Judgment (1)
            topic_id = random.choice([0, 1])
            intensity_value = random.uniform(3.0, 4.5)
        elif book == 'Leviticus':
            # Purity/Ritual (6)
            topic_id = 6
            intensity_value = random.uniform(4.5, 5.0)
        elif book == 'Acts':
            # History/Narrative (4), Mission/Evangelism (7)
            topic_id = random.choice([4, 7])
            intensity_value = random.uniform(2.5, 4.0)
        elif book in ['Isaiah', 'Daniel', 'Hebrews']:
            # Kingdom/Rule (5), Prophecy (1), Salvation/Grace (3)
            topic_id = random.choice([5, 1, 3])
            intensity_value = random.uniform(2.5, 4.0)
        else:
            # Fallback for any book not in the original hardcoded list
            topic_id = random.randint(0, num_topics - 1)
            intensity_value = random.uniform(1.0, 3.0)

        mock_data.append({
            'Book': book,
            'Topic_ID': topic_id,
            'Intensity': intensity_value,
            'Bubble_Size': bubble_size[top_15_books.index(book)],
            'Topic_Name': topics[topic_id]
        })

    df_bubble = pd.DataFrame(mock_data)

    # 4. Plotting
    plt.figure(figsize=(18, 10))

    # Map Topic_ID to a discrete color palette (using 8 colors)
    cmap = plt.cm.get_cmap('tab10', num_topics)

    # Create the scatter plot (bubble plot)
    scatter = plt.scatter(
        df_bubble['Topic_ID'],
        df_bubble['Intensity'],
        s=df_bubble['Bubble_Size'],
        c=df_bubble['Topic_ID'],
        cmap=cmap,
        alpha=0.7,
        edgecolors="k",
        linewidth=1.0
    )

    # 1. Label the bubbles with the book name
    for i, row in df_bubble.iterrows():
        plt.annotate(
            row['Book'],
            (row['Topic_ID'], row['Intensity']),
            textcoords="offset points",
            xytext=(0, 5),
            ha='center',
            fontsize=9,
            weight='bold'
        )

    # 2. Set the X-axis labels to the Topic Names
    plt.xticks(
        range(num_topics),
        topics,
        rotation=60,
        ha='right',
        fontsize=11
    )
    plt.yticks(fontsize=11)

    # 3. Set Plot Titles and Labels
    plt.title(
        'PHASE 8: Thematic Bubble Plot (Top 15 Holiness Books by Topic & Conceptual Intensity)',
        fontsize=18,
        weight='bold'
    )
    plt.xlabel('Primary Thematic Topic Area (Color-Coded)', fontsize=14)
    plt.ylabel('Thematic Intensity / Relevance (Conceptual Score 0-5)', fontsize=14)

    # 4. Add Legend for Bubble Size (Word Count)
    word_counts_sorted = sorted(word_counts_map.values())

    if len(word_counts_sorted) >= 3:
        legend_sizes_values = [
            word_counts_sorted[0],                 # Smallest
            word_counts_sorted[len(word_counts_sorted)//2], # Median
            word_counts_sorted[-1]                 # Largest
        ]
    else:
        legend_sizes_values = word_counts_sorted

    size_handles = [
        (s / word_counts.max()) * 5000 + 50
        for s in legend_sizes_values
    ]

    plt.legend(
        [plt.scatter([], [], s=s, c='gray', alpha=0.7, edgecolors="k", linewidth=1.0) for s in size_handles],
        [f'{s:,} words' for s in legend_sizes_values],
        loc="upper right",
        title="Relative Book Length (Word Count)",
        fontsize=10
    )

    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plot_filename = 'Thematic_Bubble_Plot.png'
    plt.savefig(plot_filename)
    print(f"✅ Saved Thematic Bubble Plot to {plot_filename}")
    plt.close()


In [21]:
# --- EXECUTION PHASE 6 ---
if not FINAL_ANALYSIS_DF.empty:
    create_thematic_bubble_plot(FINAL_ANALYSIS_DF, FINAL_DATA_DIR)



--- PHASE 6: Generating Thematic Bubble Plot Visualization ---


/tmp/ipython-input-1031664252.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab10', num_topics)


✅ Saved Thematic Bubble Plot to Thematic_Bubble_Plot.png


In [22]:
# ==============================================================================
# --- PHASE 7: LDA Topic Modeling on Top 15 "Holiness" Books (For completeness) ---
# ==============================================================================

def run_lda_topic_modeling(data_dir, df_holiness):
    """Performs LDA on the corpus of the top 15 books and generates a separate Word Cloud for each topic."""
    if df_holiness.empty or not data_dir: return

    print("\n--- PHASE 7: LDA Topic Modeling and Word Clouds ---")

    # 1. Define Top 15 Books
    df_top_15 = df_holiness.head(15).copy()
    top_15_books = df_top_15['Book'].tolist()

    # 2. Load Corpus
    book_texts_dict = load_book_texts(data_dir, top_15_books)
    corpus = list(book_texts_dict.values())

    if not corpus:
        print("❌ Cannot proceed with LDA: Corpus is empty.")
        return

    # 3. Preprocessing
    try:
        # Check if stopwords are available locally
        stop_words = set(stopwords.words('english'))
    except Exception:
        stop_words = set()

    custom_stopwords = ['thee', 'thou', 'thy', 'hath', 'doth', 'saying', 'said', 'shall', 'unto', 'verily', 'come', 'came', 'man', 'lord', 'god', 'hallowed', 'therefore', 'also', 'thus']
    stop_words.update(custom_stopwords)

    processed_corpus = []
    for text in corpus:
        text = text.lower()
        words = re.findall(r'\b[a-z]{3,}\b', text)
        filtered_words = [word for word in words if word not in stop_words and len(word) > 2]
        processed_corpus.append(" ".join(filtered_words))

    # 4. Create Document-Term Matrix (DTM)
    n_topics = 8
    n_top_words = 30

    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=None)
    dtm = vectorizer.fit_transform(processed_corpus)
    feature_names = vectorizer.get_feature_names_out()

    # 5. LDA Model
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        random_state=42
    )
    lda.fit(dtm)

    print(f"✅ LDA Model trained on {len(corpus)} documents with {n_topics} topics.")

    # 6. Visualization (Word Clouds)
    # Using the static topic titles provided by the user for consistency with Phase 8
    topic_titles = [
        'End Times',
        'Prophecy/Judgment',
        'Apostolic Teaching',
        'Salvation/Grace',
        'History/Narrative',
        'Kingdom/Rule',
        'Purity/Ritual',
        'Mission/Evangelism'
    ]

    colormaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'tab10', 'Pastel1', 'Dark2']

    for i, topic in enumerate(lda.components_):
        freq = {feature_names[j]: topic[j] for j in topic.argsort()[:-n_top_words - 1:-1]}

        wc = WordCloud(
            background_color='white',
            width=800,
            height=600,
            colormap=colormaps[i % len(colormaps)],
            prefer_horizontal=0.9,
            min_font_size=8
        ).generate_from_frequencies(freq)

        plt.figure(figsize=(10, 7))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis('off')

        title = topic_titles[i] if i < len(topic_titles) else f'Unnamed Topic {i+1}'
        plt.title(f"PHASE 7: Topic {i+1}: {title}", fontsize=16, weight='bold')
        plt.tight_layout()

        filename = f"Topic_{i+1}_{title.replace('&', 'and').replace(' ', '_').replace(',', '').replace('/', 'or')}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()

    print("✅ Generated Word Clouds for all 8 LDA topics.")

In [23]:
# --- EXECUTION PHASE 7 ---
if not FINAL_ANALYSIS_DF.empty:
    run_lda_topic_modeling(FINAL_DATA_DIR, FINAL_ANALYSIS_DF)


--- PHASE 7: LDA Topic Modeling and Word Clouds ---
✅ LDA Model trained on 11 documents with 8 topics.
✅ Generated Word Clouds for all 8 LDA topics.
